In [59]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 5
BS = 10
CLASSES = ["masked", "unmasked"]
DATA = []
LABELS = []
IMG_SIZE = [224, 224]
TRAIN_PATH = '/content/drive/My Drive/Colab Notebooks/dataset/Train'
TEST_PATH = '/content/drive/My Drive/Colab Notebooks/dataset/Test'

In [54]:
#import drive files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/dataset/Train', 
                                                 target_size = (224, 224), 
                                                 batch_size = BS, 
                                                 class_mode = 'categorical')

testing_set = test_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/dataset/Test', 
                                                 target_size = (224, 224), 
                                                 batch_size = BS, 
                                                 class_mode = 'categorical')

[INFO] loading images...
Found 3084 images belonging to 2 classes.
Found 749 images belonging to 2 classes.


In [61]:
#implementing the MobileNetV2 network
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3))) #concatenate the RGB channels

#head of the model will be on top of the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [62]:
for layer in baseModel.layers:
	layer.trainable = False

In [64]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the head of the network
print("[INFO] training head...")
H = model.fit(
  training_set,
	steps_per_epoch=len(testing_set) // BS,
	validation_data=testing_set,
	validation_steps=len(testing_set) // BS,
	epochs=EPOCHS)

# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testing_set, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testing_set, predIdxs,
	target_names=lb.classes_))

# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("mask_detector.model", save_format="h5")

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

[INFO] compiling model...
[INFO] training head...
Epoch 1/5
7/7 [==============================] - 11s 2s/step - loss: 0.3146 - accuracy: 0.8857 - val_loss: 0.4754 - val_accuracy: 0.7429
Epoch 2/5
7/7 [==============================] - 10s 1s/step - loss: 0.3131 - accuracy: 0.8429 - val_loss: 0.2870 - val_accuracy: 0.9429
Epoch 3/5
7/7 [==============================] - 11s 2s/step - loss: 0.2260 - accuracy: 0.9429 - val_loss: 0.3125 - val_accuracy: 0.8571
Epoch 4/5
7/7 [==============================] - 11s 2s/step - loss: 0.2231 - accuracy: 0.9429 - val_loss: 0.2727 - val_accuracy: 0.9143
Epoch 5/5
7/7 [==============================] - 10s 1s/step - loss: 0.2366 - accuracy: 0.9429 - val_loss: 0.3411 - val_accuracy: 0.8429
[INFO] evaluating network...


AttributeError: ignored